In [319]:
import os
import itertools

import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit

import importlib
importlib.reload(pipeline)
importlib.reload(setup_bestfit)
importlib.reload(bestfit)

from IPython import display

In [325]:
def fpath_ktable():
    molecule = 'h2o'
    band = '1'
    dv = .005
    nv = 200
    commitnumber = '93b34b9'
    
    atmpro = 'mls'
    tsfc = 294
    
    param = bestfit.kdist_params(molecule='h2o', band='1')
    param['molecule'] = molecule
    param['band'] = band
    param['dv'] = dv
    param['nv'] = nv
    param['commitnumber'] = commitnumber
    param['atmpro'] = atmpro
    param['tsfc'] = tsfc
    param['option_compute_ktable'] = 1
    
    fortran_dir = pipeline.get_dir_case(param, setup=setup_bestfit)

    print(pipeline.get_dir_case(param, setup=setup_bestfit))
          
    fpath_lin = os.path.join(fortran_dir, 'kg_lin.dat')
    fpath_nonlin = os.path.join(fortran_dir, 'kg_nonlin.dat')
    fpath = {'kg_lin': fpath_lin, 'kg_nonlin': fpath_nonlin}
    return fpath



def load_ktable(fpath):
    '''
    Returns k-table in the same format as in lblnew.f
    
    Parameter
    ---------
    fpath: string
        Path to the file 'ktable.dat', generated by "lblnew.f".
    ktable: array (number of (p, t) pairs, number of g-intervals)
        Absorption coefficient values calculated at selected 
        (pressure, temperature) pairs.
    '''
    df = pd.read_csv(fpath, sep=r'\s+')
    df = df.set_index(['g', 'pressure', 'temperature'])
    ng = len(df.index.levels[0].value_counts())
    nl = len(df.index.levels[1].value_counts())
    ktable = df['k'].values.reshape(ng, nl, -1)
    ktable = np.transpose(ktable, axes=(1, 2, 0))
    return ktable



def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines
    

    
def ktable_to_F77(ktable, name):
    nl, nt, ng = ktable.shape
    
    num_values_per_line = 4
    
    last_line = '/)'
    
    lines = []
    for ig in range(ng):
        for it in range(nt):
            
            first_line = '{}(:, {}, {}) = (/'.format(name, it + 1, ig + 1)
        
            lines_itg = vector_to_F77list(ktable[:, it, ig], 
                            num_values_per_line=num_values_per_line)
        
            lines_itg = [first_line] + lines_itg + [last_line]
        
            lines_itg_amp = []
            lines_itg_amp.append(6 * ' ' + lines_itg[0])
            for l in lines_itg[1:]:
                lines_itg_amp.append(5 * ' ' + '&' + l)
            
            lines.extend(lines_itg_amp)
    
    fortran = '\n'.join(lines)
    return fortran



def save_ktable(filename, s):
    clirad_dir = os.path.join('/chia_cluster/home/jackyu/radiation',
                              'clirad-lw/LW/lee_hitran2012_update/')
    
    with open(os.path.join(clirad_dir, filename),
              mode='w', encoding='utf-8') as f:
        f.write(s)
        
        
        
def script():
    for name, fpath in fpath_ktable().items():
        ktable = load_ktable(fpath)
        s = ktable_to_F77(ktable, name)
        save_ktable('{}_h2o_band01.f'.format(name), s)
    

In [326]:
script()

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band01_wn_20_340/nv_200/dv_0.005/ng_10/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__2__5/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.3_0.3_0.3__0.3_0.3__0.3_0.6_0.6_0.6_0.6/wgt_flux_2/w_diffuse_1.6_1.8_1.8__1.8_1.7__1.6_1.4_1.4_1.7_1.8/option_compute_ktable_1/crd_93b34b9


In [327]:
!ls

ch4				   n2o
co2				   Notes_lblnew_absmean.ipynb
customised_atmpro.ipynb		   notes_-_new_kdist_method.ipynb
dgdgs_to_F77.ipynb		   o3
h2o				   results.ipynb
h2o_1_co2_0_o3_1_n2o_0_ch4_0_o2_0  reverse_rrtmgs_prefs.ipynb
h2o_1_co2_1_o3_0_n2o_0_ch4_0_o2_0  rrtmg_ktable_pts_atmpro.pro
h2o_co2_band03a.ipynb		   run_lblnew_pipeline.ipynb
heuristic_rules.ipynb		   toy_kdist_1.ipynb
ktable_to_F77.ipynb		   toy_kdist.ipynb
lblnew_pipeline.ipynb
